In [1]:
import pandas as pd

In [2]:
# Read the sales data into a pandas dataframe
df = pd.ExcelFile('block_sales_2021.xlsx')
df = df.parse('Sheet1')

In [3]:
# Let us have a look at the dataframe before storing it into the database we are to create.
df.head()

,order id,date,order type,unit price,order quantity,order amount,buyer first name,buyer surname,buyer gender,buyer type
0,20210001,2021-01-04,9 inches,300,301,90300,Ekemini,Idongesit,M,Contractor
1,20210002,2021-01-04,9 inches,300,727,218100,Nnenna,Anayo,F,Supplier
2,20210003,2021-01-04,9 inches,300,775,232500,Adamu,Kabiru,M,Building Owner
3,20210004,2021-01-05,6 inches,250,177,44250,Evans,Chika,M,Building Owner
4,20210005,2021-01-05,4 inches,200,157,31400,Nnenna,Anayo,F,Supplier


In [4]:
# Let us remove spaces from column names and then proceed with the database creation.
df = df.rename(columns = {'order id':'order_id', 'order type':'order_type', 'unit price':'unit_price','order quantity':
                          'order_quantity', 'order amount':'order_amount', 'buyer first name':'buyer_first_name',
                         'buyer surname':'buyer_surname', 'buyer gender':'buyer_gender', 'buyer type':'buyer_type'})
df.head(2)

,order_id,date,order_type,unit_price,order_quantity,order_amount,buyer_first_name,buyer_surname,buyer_gender,buyer_type
0,20210001,2021-01-04,9 inches,300,301,90300,Ekemini,Idongesit,M,Contractor
1,20210002,2021-01-04,9 inches,300,727,218100,Nnenna,Anayo,F,Supplier


In [5]:
from sqlalchemy import create_engine
%load_ext sql

# Create a SQLite database
engine  = create_engine("sqlite:///block_sales_2021.db")

# Store the excel data into the database
df.to_sql('2021_orders', engine, index=False)

In [6]:
%sql sqlite:///block_sales_2021.db

In [7]:
%%sql
-- Lets check few rows of the 2021_orders table
select * from '2021_orders' limit 20

 * sqlite:///block_sales_2021.db
Done.


order_id,date,order_type,unit_price,order_quantity,order_amount,buyer_first_name,buyer_surname,buyer_gender,buyer_type
20210001,2021-01-04 00:00:00.000000,9 inches,300,301,90300,Ekemini,Idongesit,M,Contractor
20210002,2021-01-04 00:00:00.000000,9 inches,300,727,218100,Nnenna,Anayo,F,Supplier
20210003,2021-01-04 00:00:00.000000,9 inches,300,775,232500,Adamu,Kabiru,M,Building Owner
20210004,2021-01-05 00:00:00.000000,6 inches,250,177,44250,Evans,Chika,M,Building Owner
20210005,2021-01-05 00:00:00.000000,4 inches,200,157,31400,Nnenna,Anayo,F,Supplier
20210006,2021-01-05 00:00:00.000000,6 inches,250,150,37500,Adetola,Gboyega,M,Building Owner
20210007,2021-01-05 00:00:00.000000,9 inches,300,736,220800,Sambi,Mark,M,Supplier
20210008,2021-01-05 00:00:00.000000,9 inches,300,389,116700,Gabriella,Fred,F,Engineer
20210009,2021-01-06 00:00:00.000000,4 inches,200,143,28600,Evans,Chika,M,Building Owner
20210010,2021-01-06 00:00:00.000000,6 inches,250,75,18750,Adun,Olayiwola,F,Engineer


### CONVERSION OF THE DATABASE TO 1NF

- We noticed some rows contain multiple records separated by **","**. We will separate such rows and make sure each row of the table contain a single record.

- Secondly, We will make sure all entries in each column contain the same data type. We will also ensure all the columns of the table have the appropriate data types.

- Lastly, we will ensure each row is uniquely identifiable by adding a primary key column to the table.

- ##### ENSURING THERE IS NO ROW WITH MULTIPLE RECORDS

In [8]:
%%sql
select * from '2021_orders' where order_type like '%,%'

 * sqlite:///block_sales_2021.db
Done.


order_id,date,order_type,unit_price,order_quantity,order_amount,buyer_first_name,buyer_surname,buyer_gender,buyer_type
20210012,2021-01-07 00:00:00.000000,"6 inches, 4 inches","250, 200","248, 52",72400,Azuka,Chizoba,M,Engineer
20210034,2021-01-18 00:00:00.000000,"9 inches, 4 inches","300, 200","179, 100",73700,Saheed,Salaam,M,Supplier
20210079,2021-02-09 00:00:00.000000,"4 inches, 6 inches, 9 inches","200, 250, 300","31, 80, 70",68200,Sadio,Chiekh,M,Contractor
20210142,2021-03-29 00:00:00.000000,"6 inches, 4 inches","250, 200","242, 20",64500,Victor,Aki,M,Contractor
20210198,2021-04-09 00:00:00.000000,"6 inches, 9 inches","250, 300","245, 140",103250,Adun,Olayiwola,F,Engineer
20210302,2021-06-03 00:00:00.000000,"9 inches, 6 inches","300, 250","839, 50",264200,Aisha,Lateef,F,Engineer
20210303,2021-06-04 00:00:00.000000,"9 inches, 6 inches","300, 250","874, 250",312200,Sambi,Mark,M,Supplier
20210370,2021-07-14 00:00:00.000000,"4 inches, 6 inches, 9 inches","200, 250, 300","80, 200, 150",111000,Brook,Dennis,M,Building Owner
20210386,2021-07-23 00:00:00.000000,"6 inches, 4 inches","250, 200","374, 20",97500,Carter,Henry,M,Engineer
20210387,2021-07-24 00:00:00.000000,"9 inches, 6 inches","300, 250","698, 100",234400,Alex,Stephens,M,Engineer


In [9]:
%%sql
delete from '2021_orders'
where order_type like '%,%';

 * sqlite:///block_sales_2021.db
16 rows affected.


[]

In [10]:
%%sql
insert into '2021_orders' (order_id,date,order_type,unit_price,order_quantity,order_amount,buyer_first_name,
                          buyer_surname,buyer_gender,buyer_type)
values
    ('20210012', '2021-01-07 00:00:00.000000', '6 inches', 250, 248, 72400, 'Azuka', 'Chizoba', 'M', 'Engineer'),
    ('20210012', '2021-01-07 00:00:00.000000', '4 inches', 200, 52, 72400, 'Azuka', 'Chizoba', 'M', 'Engineer'),
    ('20210034', '2021-01-18 00:00:00.000000', '9 inches', 300, 179, 73700, 'Saheed', 'Salaam', 'M', 'Supplier'),
    ('20210034', '2021-01-18 00:00:00.000000', '4 inches', 200, 100, 73700, 'Saheed', 'Salaam', 'M', 'Supplier'),
    ('20210079', '2021-02-09 00:00:00.000000', '4 inches', 200, 31, 68200, 'Sadio', 'Chiekh', 'M', 'Contractor'),
    ('20210079', '2021-02-09 00:00:00.000000', '6 inches', 250, 80, 68200, 'Sadio', 'Chiekh', 'M', 'Contractor'),
    ('20210079', '2021-02-09 00:00:00.000000', '9 inches', 300, 70, 68200, 'Sadio', 'Chiekh', 'M', 'Contractor'),
    ('20210142', '2021-03-29 00:00:00.000000', '6 inches', 250, 242, 64500, 'Victor', 'Aki', 'M', 'Contractor'),
    ('20210142', '2021-03-29 00:00:00.000000', '4 inches', 200, 20, 64500, 'Victor', 'Aki', 'M', 'Contractor'),
    ('20210198', '2021-04-09 00:00:00.000000', '6 inches', '250', '245', '103250', 'Adun', 'Olayiwola', 'F', 'Engineer'),
    ('20210198', '2021-04-09 00:00:00.000000', '9 inches', '300', '140', '103250', 'Adun', 'Olayiwola', 'F', 'Engineer'),
    ('20210302', '2021-06-03 00:00:00.000000', '9 inches', 300, 839, '264200', 'Aisha', 'Lateef', 'F', 'Engineer'),
    ('20210302', '2021-06-03 00:00:00.000000', '6 inches', 250, 50, '264200', 'Aisha', 'Lateef', 'F', 'Engineer'),
    ('20210303', '2021-06-04 00:00:00.000000', '9 inches', 300, 874, 312200, 'Sambi', 'Mark', 'M', 'Supplier'),
    ('20210303', '2021-06-04 00:00:00.000000', '6 inches', 250, 250, 312200, 'Sambi', 'Mark', 'M', 'Supplier'),
    ('20210370', '2021-07-14 00:00:00.000000', '4 inches', 200, 80, 111000, 'Brook', 'Dennis', 'M', 'Building Owner'),
    ('20210370', '2021-07-14 00:00:00.000000', '6 inches', 250, 200, 111000, 'Brook', 'Dennis', 'M', 'Building Owner'),
    ('20210370', '2021-07-14 00:00:00.000000', '9 inches', 300, 150, 111000, 'Brook', 'Dennis', 'M', 'Building Owner'),
    ('20210386', '2021-07-23 00:00:00.000000', '6 inches', 250, 374, 97500, 'Carter', 'Henry', 'M', 'Engineer'),
    ('20210386', '2021-07-23 00:00:00.000000', '4 inches', 200, 20, 97500, 'Carter', 'Henry', 'M', 'Engineer'),
    ('20210387', '2021-07-24 00:00:00.000000', '9 inches', 300, 698, 234400, 'Alex', 'Stephens', 'M', 'Engineer'),
    ('20210387', '2021-07-24 00:00:00.000000', '6 inches', 250, 100, 234400, 'Alex', 'Stephens', 'M', 'Engineer'),
    ('20210428', '2021-08-06 00:00:00.000000', '4 inches', 200, 135, 57000, 'Aliyu', 'Ciroma', 'M', 'Engineer'),
    ('20210428', '2021-08-06 00:00:00.000000', '9 inches', 300, 100, 57000, 'Aliyu', 'Ciroma', 'M', 'Engineer'),
    ('20210562', '2021-10-14 00:00:00.000000', '4 inches', 200, 115, 38000, 'Chukwuma', 'Obasi', 'M', 'Building Owner'),
    ('20210562', '2021-10-14 00:00:00.000000', '6 inches', 250, 60, 38000, 'Chukwuma', 'Obasi', 'M', 'Building Owner'),
    ('20210609', '2021-11-05 00:00:00.000000', '6 inches', 250, 112, 38000, 'Sharon', 'Petterson', 'F', 'Contractor'),
    ('20210609', '2021-11-05 00:00:00.000000', '4 inches', 200, 50, 38000, 'Sharon', 'Petterson', 'F', 'Contractor'),
    ('20210683', '2021-12-10 00:00:00.000000', '4 inches', 200, 72, 60650, 'Brook', 'Dennis', 'M', 'Building Owner'),
    ('20210683', '2021-12-10 00:00:00.000000', '6 inches', 250, 65, 60650, 'Brook', 'Dennis', 'M', 'Building Owner'),
    ('20210683', '2021-12-10 00:00:00.000000', '9 inches', 300, 100, 60650, 'Brook', 'Dennis', 'M', 'Building Owner'),
    ('20210684', '2021-12-11 00:00:00.000000', '6 inches', 250, 197, 85250, 'Emeka', 'Samuel', 'M', 'Building Developer'),
    ('20210684', '2021-12-11 00:00:00.000000', '9 inches', 300, 120, 85250, 'Emeka', 'Samuel', 'M', 'Building Developer'),
    ('20210726', '2021-12-29 00:00:00.000000', '6 inches', 250, 172, 48600, 'Boye', 'Gyan', 'M', 'Building Owner'),
    ('20210726', '2021-12-29 00:00:00.000000', '4 inches', 200, 28, 48600, 'Boye', 'Gyan', 'M', 'Building Owner');

 * sqlite:///block_sales_2021.db
35 rows affected.


[]

Let us confirm that there is no row with multiple records anymore. We can use any of order_type, unit_price and order_quantity to do this since they are the 3 columns where the multiple entries are present.

In [11]:
%%sql

-- We will do this by bringing up rows with multiple records.
-- No row should come up in the result since we've eliminated all rows with multiple records.
select * from '2021_orders' where order_type like '%,%'

 * sqlite:///block_sales_2021.db
Done.


order_id,date,order_type,unit_price,order_quantity,order_amount,buyer_first_name,buyer_surname,buyer_gender,buyer_type


- ##### ENSURING ALL COLUMNS HAVE THE RIGHT DATA TYPE
- ##### ADDING A PRIMARY KEY COLUMN TO THE TABLE

In [12]:
%%sql
-- Let us first check for the data types of all the columns
pragma table_info('2021_orders')

 * sqlite:///block_sales_2021.db
Done.


cid,name,type,notnull,dflt_value,pk
0,order_id,BIGINT,0,None,0
1,date,DATETIME,0,None,0
2,order_type,TEXT,0,None,0
3,unit_price,TEXT,0,None,0
4,order_quantity,TEXT,0,None,0
5,order_amount,BIGINT,0,None,0
6,buyer_first_name,TEXT,0,None,0
7,buyer_surname,TEXT,0,None,0
8,buyer_gender,TEXT,0,None,0
9,buyer_type,TEXT,0,None,0


#### Observations
- unit_price should be in **integer** format.
- order_quantity should be in **integer** format as well.
- Lastly, all the columns with TEXT data types should be converted to VARCHAR.

In [13]:
%%sql
/*Let us rectify the data types issue and also add a primary key column to the table. We will do this by creating a new table
with the columns present in the '2021_orders' table while assigning the appropriate data types to them. We will also include a
primary key column to the table.*/

create table orders_1NF (serial_no integer primary key autoincrement, order_id varchar(10),
                         date datetime, order_type varchar(10), unit_price integer,
                         order_quantity integer, order_amount integer, buyer_first_name varchar(50),
                         buyer_surname varchar(50), buyer_gender varchar(1), buyer_type varchar(50));

 * sqlite:///block_sales_2021.db
Done.


[]

In [14]:
%%sql

insert into orders_1NF (order_id, date, order_type, unit_price, order_quantity, order_amount, buyer_first_name,
                         buyer_surname, buyer_gender, buyer_type)
select * from '2021_orders'
order by date;

 * sqlite:///block_sales_2021.db
747 rows affected.


[]

Let us have a look at the table so we can confirm the changes in data types and also the primary key inclusion.

In [15]:
%%sql
select * from orders_1NF

 * sqlite:///block_sales_2021.db
Done.


serial_no,order_id,date,order_type,unit_price,order_quantity,order_amount,buyer_first_name,buyer_surname,buyer_gender,buyer_type
1,20210001,2021-01-04 00:00:00.000000,9 inches,300,301,90300,Ekemini,Idongesit,M,Contractor
2,20210002,2021-01-04 00:00:00.000000,9 inches,300,727,218100,Nnenna,Anayo,F,Supplier
3,20210003,2021-01-04 00:00:00.000000,9 inches,300,775,232500,Adamu,Kabiru,M,Building Owner
4,20210004,2021-01-05 00:00:00.000000,6 inches,250,177,44250,Evans,Chika,M,Building Owner
5,20210005,2021-01-05 00:00:00.000000,4 inches,200,157,31400,Nnenna,Anayo,F,Supplier
6,20210006,2021-01-05 00:00:00.000000,6 inches,250,150,37500,Adetola,Gboyega,M,Building Owner
7,20210007,2021-01-05 00:00:00.000000,9 inches,300,736,220800,Sambi,Mark,M,Supplier
8,20210008,2021-01-05 00:00:00.000000,9 inches,300,389,116700,Gabriella,Fred,F,Engineer
9,20210009,2021-01-06 00:00:00.000000,4 inches,200,143,28600,Evans,Chika,M,Building Owner
10,20210010,2021-01-06 00:00:00.000000,6 inches,250,75,18750,Adun,Olayiwola,F,Engineer


In [16]:
%%sql
pragma table_info (orders_1NF)

 * sqlite:///block_sales_2021.db
Done.


cid,name,type,notnull,dflt_value,pk
0,serial_no,integer,0,None,1
1,order_id,varchar(10),0,None,0
2,date,datetime,0,None,0
3,order_type,varchar(10),0,None,0
4,unit_price,integer,0,None,0
5,order_quantity,integer,0,None,0
6,order_amount,integer,0,None,0
7,buyer_first_name,varchar(50),0,None,0
8,buyer_surname,varchar(50),0,None,0
9,buyer_gender,varchar(1),0,None,0


**We have now successfully converted the "2022_orders" table to the first normal form (I.E 1NF). The resulting table is "orders_1NF".** 

**Next, we will be converting "orders_1NF" to the second normal form.**